In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import sqlite3
import json
import os
import yaml
import time
from urllib.parse import urlparse, urljoin # Добавляем импорт urlparse

In [2]:
# Настройка опций Chrome для обхода SSL и скрытия Selenium
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument(f"user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--allow-insecure-localhost")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)

In [3]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

In [4]:
conn = sqlite3.connect('database/prosport.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()
cursor.execute("SELECT id, name, url, slug FROM sports")
db_sports_data = cursor.fetchall()

In [5]:
filtred_links =db_sports_data

In [6]:
def insert_tournament(cursor, name, url, sport_id, season=None, tournaments_url=None):
    """
    Вставляет новый турнир (сезон турнира) в БД или возвращает ID существующего.
    'url' - URL тега в новостях.
    'tournaments_url' - URL страницы данных турнира.
    """
    try:
        # tournaments_url теперь является уникальным ключом для каждой записи сезона
        cursor.execute("INSERT OR IGNORE INTO tournaments (name, url, sport_id, season, tournaments_url) VALUES (?, ?, ?, ?, ?)",
                       (name, url, sport_id, season, tournaments_url))
        if cursor.lastrowid:
            print(f"    ✅ Добавлен турнир: {name} (ID: {cursor.lastrowid}, Сезон: {season}, URL тега: {url}, URL данных: {tournaments_url}) для спорта ID: {sport_id}) в БД.")
            return cursor.lastrowid
        else:
            # Если запись уже существует (по 'tournaments_url'), проверяем и обновляем 'name', 'url', 'season'
            cursor.execute("SELECT id, name, url, season FROM tournaments WHERE tournaments_url = ?", (tournaments_url,))
            existing_data = cursor.fetchone()
            if existing_data:
                existing_id = existing_data['id']
                existing_name = existing_data['name']
                existing_url = existing_data['url']
                existing_season = existing_data['season']
                
                update_needed = False
                if existing_name != name:
                    update_needed = True
                if existing_url != url:
                    update_needed = True
                if existing_season != season:
                    update_needed = True

                if update_needed:
                    cursor.execute("UPDATE tournaments SET name = ?, url = ?, season = ? WHERE id = ?", (name, url, season, existing_id))
                    print(f"    🔄 Обновлен турнир '{existing_name}' (ID: {existing_id}). Имя: {existing_name} -> {name}, URL тега: {existing_url} -> {url}, Сезон: {existing_season} -> {season}) в БД.")
                else:
                    print(f"    ℹ️ Турнир '{name}' уже существует (ID: {existing_id}). Данные не изменились.")
                return existing_id
            else:
                print(f"    ⚠️ Не удалось добавить турнир '{name}'.")
                return None
    except sqlite3.Error as e:
        print(f"❌ Ошибка при вставке/обновлении турнира '{name}': {e}")
        return None
# === КОНЕЦ Вспомогательной функции ===

In [7]:
print(filtred_links)

[<sqlite3.Row object at 0x000001EA1788B340>, <sqlite3.Row object at 0x000001EA1780B5E0>, <sqlite3.Row object at 0x000001EA17B664D0>, <sqlite3.Row object at 0x000001EA17B66530>, <sqlite3.Row object at 0x000001EA17B66560>, <sqlite3.Row object at 0x000001EA17B665C0>, <sqlite3.Row object at 0x000001EA17B665F0>, <sqlite3.Row object at 0x000001EA17B66350>, <sqlite3.Row object at 0x000001EA17B66320>, <sqlite3.Row object at 0x000001EA17B65600>, <sqlite3.Row object at 0x000001EA17B666B0>]


In [8]:
all_tournaments = []

stat_slug_overrides = {
    "boxing": "mma", 
}

for sport_data in filtred_links: 
    sport_id = sport_data['id']      
    sport_name = sport_data['name']  
    link = sport_data['url']         
    original_sport_slug = sport_data['slug'] 

    effective_stat_slug = original_sport_slug
    if "/boxing/" in link: 
        effective_stat_slug = "mma" 

    print(f"\n--- Обработка вида спорта: {sport_name} (ID: {sport_id}, URL: {link}, Original Slug: {original_sport_slug}, Effective Stat Slug: {effective_stat_slug}) ---")

    try:
        stat_url = f"https://www.championat.com/stat/{effective_stat_slug}/"
        print(f"  Переходим на страницу статистики/турниров: {stat_url}")

        driver.get(stat_url)
        
        tournament_links = []
        li_elements = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "li.block__title")))
        
        for li in li_elements:
            a_tags = li.find_elements(By.CSS_SELECTOR, "a.block__more")
            for a in a_tags:
                href = a.get_attribute("href")
                if href:
                    tournament_links.append(href)
        
        for t_link in tournament_links:
            driver.get(t_link)
            tables = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "mc-sport")))
            for table in tables:
                a_tags = table.find_elements(By.CSS_SELECTOR, "a.table-item")
                for a in a_tags:
                    name = ""
                    href_tournament_data = "" 
                    
                    raw_href_from_get_attribute = a.get_attribute("href")
                    raw_href_from_js = ""

                    if raw_href_from_get_attribute:
                        href_tournament_data = urljoin(driver.current_url, raw_href_from_get_attribute)
                    else:
                        try:
                            raw_href_from_js = driver.execute_script("return arguments[0].getAttribute('href');", a)
                            if raw_href_from_js:
                                href_tournament_data = urljoin(driver.current_url, raw_href_from_js)
                        except Exception as js_href_e:
                            print(f"    ❌ Ошибка JavaScript при получении href для элемента: {js_href_e}. HTML: {a.get_attribute('outerHTML')[:200]}...")
                            href_tournament_data = "" 

                    try:
                        name_span_element = WebDriverWait(a, 5).until(
                            EC.visibility_of_element_located((By.CSS_SELECTOR, "span.table-item__name"))
                        )
                        name = name_span_element.text.strip()
                    except (NoSuchElementException, TimeoutException):
                        pass 

                    if not name:
                        name = a.text.strip()
                    
                    if not name:
                        try:
                            name = driver.execute_script("return arguments[0].textContent;", a).strip()
                        except Exception as js_e:
                            print(f"    ❌ Ошибка JavaScript при получении textContent для {href_tournament_data}: {js_e}")
                            name = "" 
                    
                    if not name: 
                        print(f"    ⚠️ Пропущен турнир из-за отсутствия имени после всех попыток. "
                              f"URL: {href_tournament_data}. "
                              f"HTML элемента (начало): {a.get_attribute('outerHTML')[:200]}..."
                              f" (конец): {a.get_attribute('outerHTML')[-100:]}") 
                        continue 
                    
                    if href_tournament_data: 
                        # --- ВОЗВРАЩАЕМ ЛОГИКУ ФОРМИРОВАНИЯ news_tag_url_for_db ИЗ href_tournament_data ---
                        parsed_data_url = urlparse(href_tournament_data)
                        base_domain_url = f"{parsed_data_url.scheme}://{parsed_data_url.netloc}"
                        
                        path_segments_data = [s for s in parsed_data_url.path.split('/') if s]
                        news_tag_path = ""
                        if "tournament" in path_segments_data:
                            idx = path_segments_data.index("tournament")
                            news_tag_path = "/" + "/".join(path_segments_data[:idx])
                        elif len(path_segments_data) >= 2:
                            news_tag_path = "/" + "/".join(path_segments_data[:2])
                        elif len(path_segments_data) == 1:
                            news_tag_path = "/" + path_segments_data[0]
                        
                        news_tag_url_for_db = urljoin(base_domain_url, news_tag_path + ".html")
                        # --- КОНЕЦ ВОЗВРАЩЕННОЙ ЛОГИКИ ---

                        insert_tournament(
                            cursor=cursor,
                            name=name,
                            url=news_tag_url_for_db, # Теперь здесь будет преобразованный URL тега
                            sport_id=sport_id,
                            season=None, 
                            tournaments_url=href_tournament_data 
                        )
                    else:
                        print(f"    ⚠️ Пропущен турнир из-за отсутствия URL (даже после всех попыток извлечения): Name='{name}'. HTML: {a.get_attribute('outerHTML')[:200]}...") 
            time.sleep(1) 
    except TimeoutException as te:
        print(f"❌ Ошибка таймаута при обработке {link} (Stat URL: {stat_url}): {te}. Возможно, элементы не появились или страница не загрузилась.")
    except NoSuchElementException as nse:
        print(f"❌ Ошибка: Элемент не найден на странице {link} (Stat URL: {stat_url}): {nse}. Проверьте селекторы.")
    except Exception as e:
        print(f"❌ Непредвиденная ошибка при обработке {link} (Stat URL: {stat_url}): {e}")


--- Обработка вида спорта: Футбол (ID: 1, URL: https://www.championat.com/football/, Original Slug: football, Effective Stat Slug: football) ---
  Переходим на страницу статистики/турниров: https://www.championat.com/stat/football/


    ℹ️ Турнир 'Австралия — А-лига' уже существует (ID: 661). Данные не изменились.
    ℹ️ Турнир 'Австрия — Бундеслига' уже существует (ID: 1). Данные не изменились.
    ℹ️ Турнир 'Кубок Австрии' уже существует (ID: 2). Данные не изменились.
    ℹ️ Турнир 'Азербайджан — Премьер-лига' уже существует (ID: 3). Данные не изменились.
    ℹ️ Турнир 'Албания — Суперлига' уже существует (ID: 4). Данные не изменились.
    ℹ️ Турнир 'Суперкубок Англии — 2025' уже существует (ID: 5). Данные не изменились.
    ℹ️ Турнир 'Англия — Премьер-лига' уже существует (ID: 6). Данные не изменились.
    ℹ️ Турнир 'Англия — Кубок лиги' уже существует (ID: 7). Данные не изменились.
    ℹ️ Турнир 'Англия — Чемпионшип' уже существует (ID: 8). Данные не изменились.
    ℹ️ Турнир 'Летняя серия Премьер-лиги' уже существует (ID: 9). Данные не изменились.
    ℹ️ Турнир 'Аргентина — Примера' уже существует (ID: 10). Данные не изменились.
    ℹ️ Турнир 'Кубок Аргентины 2025' уже существует (ID: 11). Данные не изменилис

In [9]:
conn.commit()
print("\n✅ Все изменения зафиксированы в базе данных.")


✅ Все изменения зафиксированы в базе данных.


In [10]:
conn.close()
print("✅ Соединение с базой данных закрыто.")

✅ Соединение с базой данных закрыто.


In [11]:
# import pandas as pd
# pd.DataFrame(all_tournaments)

In [12]:
driver.close()